In [2]:
import time
import json
import ast
import os
import datetime
import io
from collections import defaultdict

import imageio
import boto3
import pandas as pd
import numpy as np
import imageio
import matplotlib.pyplot as plt
# import seaborn as sns

from brtdevkit.core.db.athena import AthenaClient
from brtdevkit.data import Dataset
from timezonefinder import TimezoneFinderL
import pytz

from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import imageids_to_dataset
from aletheia_dataset_creator.config.dataset_config import LEFT_CAMERAS, ALL_CAMERA_PAIRS_LIST
%matplotlib inline
pd.set_option('display.max_rows', 500)
import json
import cv2
import os
import multiprocessing as mp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dl.utils.io_utils import normalize_image

%matplotlib inline
from cv.core.image_quality_server_side import ImageQuality

# from cv.core.image_quality_server_side_halo import ImageQuality
from dl.utils.config import DEFAULT_TONEMAP_PARAMS

In [17]:
directory = '/data2/jupiter/datasets/halo_rgb_stereo_test_v6_0/'
csv_name = 'master_annotations.csv'

In [22]:
df = pd.read_csv(directory + csv_name)

/home/alex.li/miniconda3/envs/cvml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (0,1,22,79,90,91,92,93,94,96,97,99,100,101,102,103,104,105,106,107,117,150,154,155,160,171,174,196,197,199,200,201,202,227,228,235,236,250,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,271,275,276,277,280,284) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [36]:
list(df.groupby(['camera_location', 'robot_name']).groups.keys())

[('I01', 'halohitchhiker_103'),
 ('I01', 'halohitchhiker_113'),
 ('I01', 'halohitchhiker_123'),
 ('I02', 'halohitchhiker_103'),
 ('I02', 'halohitchhiker_113'),
 ('I02', 'halohitchhiker_123'),
 ('T01', 'halohitchhiker_101'),
 ('T01', 'halohitchhiker_111'),
 ('T01', 'halohitchhiker_121'),
 ('T01', 'halohitchhiker_161'),
 ('T01', 'halohitchhiker_201'),
 ('T01', 'halohitchhiker_211'),
 ('T01', 'halohitchhiker_221'),
 ('T02', 'halohitchhiker_101'),
 ('T02', 'halohitchhiker_111'),
 ('T02', 'halohitchhiker_121'),
 ('T02', 'halohitchhiker_161'),
 ('T02', 'halohitchhiker_201'),
 ('T02', 'halohitchhiker_211'),
 ('T02', 'halohitchhiker_221'),
 ('T05', 'halohitchhiker_101'),
 ('T05', 'halohitchhiker_111'),
 ('T05', 'halohitchhiker_121'),
 ('T05', 'halohitchhiker_161'),
 ('T05', 'halohitchhiker_201'),
 ('T05', 'halohitchhiker_211'),
 ('T05', 'halohitchhiker_221'),
 ('T06', 'halohitchhiker_101'),
 ('T06', 'halohitchhiker_111'),
 ('T06', 'halohitchhiker_121'),
 ('T06', 'halohitchhiker_161'),
 ('T06',

{('I01', 'halohitchhiker_103'): [27057, 27059, 27060, 27061, 27062, 27063, 27064, 27065, 27066, 27067, 27068, 27069, 27070, 27071, 27072, 27073, 27074, 27075, 27076, 27077, 27078, 27079, 27080, 27081, 27082, 27083, 27084, 27085, 27086, 27087, 27088, 27089, 27090, 27091, 27092, 27093, 27094, 27095, 27096, 27097, 27098, 27099, 27100, 27101, 27102, 27103, 27104, 27105, 27106, 27107, 27108, 27109, 27110, 27111, 27112, 27113, 27114, 27115, 27116, 27117, 27118, 27119, 27120, 27121, 27122, 27123, 27124, 27125, 27126, 27127, 27128, 27129, 27130, 27131, 27132, 27133, 27134, 27135, 27136, 27137, 27138, 27139, 27140, 27141, 27142, 27143, 27144, 27145, 27146, 27147, 27148, 27149, 27150, 27151, 27152, 27153, 27154, 27155, 27156, 27157, ...], ('I01', 'halohitchhiker_113'): [27237, 27254, 27255, 27256, 27257, 27258, 27259, 27260, 27261, 27262, 27263, 27264, 27265, 27266, 27267, 27268, 27269, 27270, 27271, 27272, 27273, 27274, 27275, 27276, 27277, 27278, 27279, 27281, 27287, 27310, 27311, 27312, 27313